In [10]:


from alpaca.data.historical import CryptoHistoricalDataClient

client = CryptoHistoricalDataClient()

from dotenv import load_dotenv

load_dotenv()

from datetime import datetime
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
import os
import alpaca_trade_api as tradeapi
import pandas as pd
from alpaca_trade_api.rest import TimeFrame
import requests

In [ ]:



# Creating request object
request_params = CryptoBarsRequest(
  symbol_or_symbols=["BTC/USD"],
  timeframe=TimeFrame.Day,
  start=datetime(2025, 8, 1),
  end=datetime(2025, 8, 7)
)


In [7]:
# Retrieve daily bars for Bitcoin in a DataFrame and printing it
btc_bars = client.get_crypto_bars(request_params)

# Convert to dataframe
btc_bars.df


open         high            low  \
symbol  timestamp                                                           
BTC/USD 2025-08-01 00:00:00+00:00  115803.863  116097.5660  112727.079803   
        2025-08-02 00:00:00+00:00  113208.800  114111.1000  112054.800000   
        2025-08-03 00:00:00+00:00  112480.352  114813.3000  111981.015000   
        2025-08-04 00:00:00+00:00  114288.156  115688.9775  114133.770000   
        2025-08-05 00:00:00+00:00  114978.780  115158.8000  112656.475000   
        2025-08-06 00:00:00+00:00  114151.448  115774.4000  113340.656000   
        2025-08-07 00:00:00+00:00  114982.866  117704.6350  114284.981000   

                                         close    volume  trade_count  \
symbol  timestamp                                                       
BTC/USD 2025-08-01 00:00:00+00:00  113277.9550  7.508886        503.0   
        2025-08-02 00:00:00+00:00  112572.5650  2.603162        366.0   
        2025-08-03 00:00:00+00:00  114237.0950  0.652422        257.0   
        2025-08-04 00:00:00+00:00  115084.3735  0.902563        363.0   
        2025-08-05 00:00:00+00:00  114172.1930  1.902997        288.0   
        2025-08-06 00:00:00+00:00  115059.2600  3.119831        408.0   
        2025-08-07 00:00:00+00:00  117408.1775  3.132966        700.0   

                                            vwap  
symbol  timestamp                                 
BTC/USD 2025-08-01 00:00:00+00:00  114771.722096  
        2025-08-02 00:00:00+00:00  113426.594877  
        2025-08-03 00:00:00+00:00  113431.868755  
        2025-08-04 00:00:00+00:00  114778.766637  
        2025-08-05 00:00:00+00:00  113465.965432  
        2025-08-06 00:00:00+00:00  115114.246087  
        2025-08-07 00:00:00+00:00  116465.353494

In [2]:
target_stocks = ["APPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META"]

In [11]:
headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": os.getenv('APCA_API_KEY_ID'),
    "APCA-API-SECRET-KEY": os.getenv('APCA_API_SECRET_KEY')
}

print(os.getenv('APCA_API_KEY_ID'))
print(os.getenv('APCA_API_SECRET_KEY'))

def get_data1_now(stock, currency = "USD"):
    url = "https://data.alpaca.markets/v2/stocks/bars/latest?symbols={}&feed=iex&currency={}".format(stock, currency)
    response = requests.get(url, headers = headers)
    return response

def get_data_target_stocks():
    result = []
    for stock in target_stocks:
        result.append(get_data1_now(stock))
    return result





PKNCBYPWH6MG04CKP4PJ
Sz4WemL9iDfNNbRl242rE3Zc8lqCuc5A6UXU0bpg


In [13]:
for u in get_data_target_stocks():
    print(u.text)

{"bars":{}}
{"bars":{"MSFT":{"c":520.8,"h":520.8,"l":519.88,"n":203,"o":519.88,"t":"2025-08-07T19:59:00Z","v":7564,"vw":520.28821}}}
{"bars":{"GOOGL":{"c":196.56,"h":196.56,"l":196.36,"n":286,"o":196.38,"t":"2025-08-07T19:59:00Z","v":17793,"vw":196.431541}}}
{"bars":{"AMZN":{"c":223.18,"h":223.18,"l":222.995,"n":387,"o":223.06,"t":"2025-08-07T19:59:00Z","v":25373,"vw":223.103363}}}
{"bars":{"NVDA":{"c":180.85,"h":180.85,"l":180.49,"n":331,"o":180.49,"t":"2025-08-07T19:59:00Z","v":30182,"vw":180.75659}}}
{"bars":{"META":{"c":761.57,"h":761.935,"l":761.46,"n":273,"o":761.565,"t":"2025-08-07T19:59:00Z","v":8977,"vw":761.641976}}}


In [22]:
import os
import requests
import json
import sqlite3
from dotenv import load_dotenv

# --- Configuration ---
load_dotenv() # Load environment variables from .env file

# Your list of stocks
TARGET_STOCKS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META']
DB_FILE = 'stock_data.db' # The name of our SQLite database file

# Get API keys from environment
API_KEY = os.getenv('APCA_API_KEY_ID')
SECRET_KEY = os.getenv('APCA_API_SECRET_KEY')

# Alpaca API headers
HEADERS = {
    "APCA-API-KEY-ID": API_KEY,
    "APCA-API-SECRET-KEY": SECRET_KEY
}

def setup_database():
    """Creates the SQLite database and the stock_bars table if they don't exist."""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    # The SQL query to create our table.
    # "IF NOT EXISTS" prevents an error if the table is already there.
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS latest_stock_bars_exact (
            symbol TEXT NOT NULL,
            timestamp DATETIME NOT NULL,
            open REAL NOT NULL,
            high REAL NOT NULL,
            low REAL NOT NULL,
            close REAL NOT NULL,
            volume INTEGER NOT NULL,
            trade_count INTEGER NOT NULL,
            vwap REAL NOT NULL,
            PRIMARY KEY (symbol, timestamp)
        )
    ''')
    
    conn.commit()
    conn.close()
    print("Database setup complete.")

def fetch_and_store_data():
    """Fetches latest bar data for all target stocks and stores it in the database."""
    
    # More efficient: Fetch all symbols in a single API call
    symbols_str = ",".join(TARGET_STOCKS)
    url = f"https://data.alpaca.markets/v2/stocks/bars/latest?symbols={symbols_str}&feed=iex"
    
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        
        data = response.json()
        bars = data.get('bars', {})
        
        if not bars:
            print("No bar data received from API.")
            return

        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()

        for symbol, bar_data in bars.items():
            # The SQL query to insert data.
            # "INSERT OR IGNORE" is key: if a row with the same primary key (symbol, timestamp)
            # already exists, it will be silently ignored, preventing duplicates.
            sql = '''
                INSERT OR IGNORE INTO latest_stock_bars_exact (symbol, timestamp, open, high, low, close, volume, trade_count, vwap)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            '''
            
            # Prepare the data tuple in the correct order
            data_tuple = (
                symbol,
                bar_data['t'],
                bar_data['o'],
                bar_data['h'],
                bar_data['l'],
                bar_data['c'],
                bar_data['v'],
                bar_data['n'],
                bar_data['vw']
            )
            
            cursor.execute(sql, data_tuple)
            print(f"Stored/updated data for {symbol} at {bar_data['t']}")

        conn.commit()
        conn.close()

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Alpaca API: {e}")
    except json.JSONDecodeError:
        print("Error decoding JSON from API response.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    print("--- Starting Data Logging Script ---")
    setup_database()
    fetch_and_store_data()
    print("--- Script Finished ---")

--- Starting Data Logging Script ---
Database setup complete.
Stored/updated data for META at 2025-08-07T19:59:00Z
Stored/updated data for NVDA at 2025-08-07T19:59:00Z
Stored/updated data for AMZN at 2025-08-07T19:59:00Z
Stored/updated data for GOOGL at 2025-08-07T19:59:00Z
Stored/updated data for MSFT at 2025-08-07T19:59:00Z
Stored/updated data for AAPL at 2025-08-07T19:59:00Z
--- Script Finished ---


In [23]:
import sqlite3
import pandas as pd
import os

# --- Configuration ---
DB_FILE = 'stock_data.db' # The database file created by data_logger.py

def view_database_contents():
    """Connects to the SQLite database and prints its contents using Pandas."""
    
    # First, check if the database file actually exists
    if not os.path.exists(DB_FILE):
        print(f"Error: Database file '{DB_FILE}' not found.")
        print("Please run the data_logger.py script first to create and populate it.")
        return

    conn = None # Initialize conn to None
    try:
        # Establish a connection to the SQLite database
        conn = sqlite3.connect(DB_FILE)
        
        print(f"--- Successfully connected to {DB_FILE} ---")
        
        # --- Example 1: View the LATEST 10 entries in the entire table ---
        print("\n=======================================================")
        print("           Most Recent 10 Entries Overall")
        print("=======================================================")
        
        query_all = "SELECT * FROM latest_stock_bars_exact ORDER BY timestamp DESC LIMIT 10"
        df_all = pd.read_sql_query(query_all, conn)
        
        if df_all.empty:
            print("The database is empty. No data to display.")
        else:
            print(df_all.to_string()) # .to_string() ensures all columns are shown


        # --- Example 2: View all data for a SINGLE stock (e.g., AAPL) ---
        print("\n\n=======================================================")
        print("              All Stored Data for AAPL")
        print("=======================================================")
        
        symbol_to_check = 'AAPL'
        query_symbol = f"SELECT * FROM latest_stock_bars_exact WHERE symbol = '{symbol_to_check}' ORDER BY timestamp DESC"
        df_symbol = pd.read_sql_query(query_symbol, conn)
        
        if df_symbol.empty:
            print(f"No data found for symbol: {symbol_to_check}")
        else:
            print(df_symbol.to_string())
            
            
        # --- Example 3: View a count of entries per stock ---
        print("\n\n=======================================================")
        print("            Total Entries Logged Per Stock")
        print("=======================================================")
        
        query_count = "SELECT symbol, COUNT(*) as entry_count FROM latest_stock_bars_exact GROUP BY symbol"
        df_count = pd.read_sql_query(query_count, conn)
        
        if df_count.empty:
             print("No data to count.")
        else:
            print(df_count.to_string())


    except sqlite3.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        # Ensure the database connection is closed, even if an error occurred
        if conn:
            conn.close()
            print("\n--- Database connection closed ---")

if __name__ == "__main__":
    view_database_contents()

--- Successfully connected to stock_data.db ---

           Most Recent 10 Entries Overall
  symbol             timestamp     open     high      low   close  volume  trade_count        vwap
0   AMZN  2025-08-07T19:59:00Z  223.060  223.180  222.995  223.18   25373          387  223.103363
1  GOOGL  2025-08-07T19:59:00Z  196.380  196.560  196.360  196.56   17793          286  196.431541
2   MSFT  2025-08-07T19:59:00Z  519.880  520.800  519.880  520.80    7564          203  520.288210
3   AAPL  2025-08-07T19:59:00Z  219.845  220.170  219.845  220.17   23230          299  220.001071
4   META  2025-08-07T19:59:00Z  761.565  761.935  761.460  761.57    8977          273  761.641976
5   NVDA  2025-08-07T19:59:00Z  180.490  180.850  180.490  180.85   30182          331  180.756590


              All Stored Data for AAPL
  symbol             timestamp     open    high      low   close  volume  trade_count        vwap
0   AAPL  2025-08-07T19:59:00Z  219.845  220.17  219.845  220.17   23230     

In [ ]:



# --- Configuration ---
# List of stocks we want to focus on for our trading agent.
STOCKS_TO_WATCH = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META']

def get_alpaca_api():
    """
    Instantiates the Alpaca API client.
    It automatically looks for the API keys in your environment variables.
    """
    # Make sure to use the paper trading URL for testing
    base_url = 'https://paper-api.alpaca.markets'
    api = tradeapi.REST(base_url=base_url)
    return api

def fetch_all_data(api, stocks):
    """
    Fetches snapshot, historical bars, and news for a list of stocks.
    Uses get_snapshot() which is compatible with free data plans.
    """
    print("--- Starting Data Fetch for Trading Agent ---")
    
    # Get snapshots for all stocks at once for efficiency
    try:
        snapshots = api.get_snapshots(stocks)
    except Exception as e:
        print(f"Could not fetch snapshots. Error: {e}")
        snapshots = {} # Ensure snapshots is a dict to avoid further errors

    for symbol in stocks:
        print(f"\n==================== {symbol} ====================")
        try:
            snapshot = snapshots.get(symbol)

            # 1. Fetch the Latest Data from the Snapshot
            if snapshot:
                latest_trade = snapshot.latest_trade
                latest_quote = snapshot.latest_quote
                print(f"Latest Trade Price: ${latest_trade.p:.2f}")
                print(f"Latest Quote (Bid/Ask): ${latest_quote.bp:.2f} / ${latest_quote.ap:.2f}")
            else:
                print("Could not retrieve snapshot for this symbol.")


            # 2. Fetch Historical Data (Hourly Bars for Technical Analysis)
            # This function remains the same and generally works with free plans.
            bars = api.get_bars(
                symbol,
                TimeFrame.Hour,
                limit=100,
                adjustment='raw'
            )
            bars_df = pd.DataFrame(
                [bar.__dict__['_raw'] for bar in bars],
                columns=['t', 'o', 'h', 'l', 'c', 'v']
            ).set_index('t')
            bars_df.index = pd.to_datetime(bars_df.index)
            print("\nRecent Hourly Data (last 5 hours):")
            print(bars_df.tail(5))


            # 3. Fetch Recent News (For Sentiment Analysis)
            news = api.get_news(symbol, limit=5)
            print("\nRecent News:")
            if news:
                for article in news:
                    print(f"  - [{article.created_at.date()}] {article.headline}")
            else:
                print("  - No recent news found.")

        except Exception as e:
            # This will catch errors for individual stock processing
            print(f"Could not process data for {symbol}. Error: {e}")

    print("\n--- Data Fetch Complete ---")

if __name__ == '__main__':
    # This block runs when you execute the script directly
    alpaca_api = get_alpaca_api()
    fetch_all_data(alpaca_api, STOCKS_TO_WATCH)

--- Starting Data Fetch for Trading Agent ---
Could not fetch snapshots. Error: 403 Client Error: Forbidden for url: https://data.alpaca.markets/v2/stocks/snapshots?symbols=AAPL,MSFT,GOOGL,AMZN,NVDA,META

==================== AAPL ====================
Could not retrieve snapshot for this symbol.
Could not process data for AAPL. Error: 403 Client Error: Forbidden for url: https://data.alpaca.markets/v2/stocks/AAPL/bars?timeframe=1Hour&adjustment=raw&limit=100

==================== MSFT ====================
Could not retrieve snapshot for this symbol.
Could not process data for MSFT. Error: 403 Client Error: Forbidden for url: https://data.alpaca.markets/v2/stocks/MSFT/bars?timeframe=1Hour&adjustment=raw&limit=100

==================== GOOGL ====================
Could not retrieve snapshot for this symbol.
Could not process data for GOOGL. Error: 403 Client Error: Forbidden for url: https://data.alpaca.markets/v2/stocks/GOOGL/bars?timeframe=1Hour&adjustment=raw&limit=100

===============